In [19]:
import numpy as np
class HMM():
    def __init__(self, Observations, Transition, Emission, Initial_distribution):
        self.Observations = Observations
        self.Transition = Transition
        self.Emission = Emission
        self.Initial_distribution = Initial_distribution
    
    def forward(self):
        alpha = np.zeros((self.Observations.shape[0], self.Transition.shape[0]))
        
        
        for i in range(self.Observations.shape[0]):
            sequence_val = self.Observations[i]
            for j in range(self.Transition.shape[0]):
                if (i == 0):
                    alpha[i, j] = self.Initial_distribution[j] * self.Emission[j,sequence_val]
                else:
                    values = [alpha[i - 1,k] * self.Emission[j,sequence_val] * self.Transition[k,j] for k in
                              range(self.Transition.shape[0])]
                    alpha[i,j] = sum(values)
        return alpha
    
    def backward(self):
        beta = np.zeros((self.Observations.shape[0], self.Transition.shape[0]))
        for i in range(1,1+self.Observations.shape[0]):
            for j in range(self.Transition.shape[0]):
                if (-i == -1):
                    beta[-i, j] = 1
                else:
                    values = [beta[-i+1,k] * self.Emission[k,self.Observations[-i+1]] * self.Transition[k,j] for k in range(self.Transition.shape[0])]
                    beta[-i,j] = sum(values)
        return beta
    
    def gamma_comp(self, alpha, beta):
        
        gamma = np.zeros((self.Observations.shape[0], self.Transition.shape[0]))
        alpha_end = sum(alpha[-1,:])
        print(alpha_end)
        for i in range(self.Observations.shape[0]):
            for j in range(self.Transition.shape[0]):
                gamma[i, j] = (alpha[i, j] * beta[i, j]) / alpha_end
        return gamma
    
    def xi_comp(self, alpha, beta, gamma):
        
        xi = np.zeros((self.Observations.shape[0]-1, self.Transition.shape[0], 
self.Transition.shape[0]))
        alpha_end = sum(alpha[-1,:])
        for i in range(self.Observations.shape[0]-1):
            for j in range(self.Transition.shape[0]):
                for k in range(self.Transition.shape[0]):
                    xi[i,j,k] = ( alpha[i,j] * beta[i+1,k] * self.Transition[k,j] * self.Emission[k,self.Observations[i+1]] )\
                    / alpha_end        
        return xi
    def update(self, alpha, beta, gamma, xi):
        new_init_state = np.zeros_like(self.Initial_distribution)
        T_prime = np.zeros_like(self.Transition)
        M_prime = np.zeros_like(self.Emission)
        
        for i in range(self.Transition.shape[0]):
            for j in range(self.Transition.shape[0]):
                for t in range(self.Observations.shape[0]-1):
                    T_prime[i,j] = T_prime[i,j] + xi[t,i,j]

                denomenator = [xi[t_x, i , i_x] for t_x in range(self.Observations.shape[0] - 1) for i_x in range(self.Transition.shape[0])]
                denomenator = sum(denomenator)

                if (denomenator == 0):
                    T_prime[i,j] = 0
                else:
                    T_prime[i,j] = T_prime[i,j]/denomenator
                    
        for i in range(self.Transition.shape[0]): 
            for j in range(self.Emission.shape[1]): 
                indices = [idx for idx in range(len(self.Observations)) if(self.Observations[idx] == j)]
                numerator_b = sum( gamma[indices,i] )
                denomenator_b = sum( gamma[:,i] )

                if (denomenator_b == 0):
                    M_prime[i,j] = 0
                else:
                    M_prime[i,j] = numerator_b / denomenator_b
        new_init_state = gamma[0,:]
        return T_prime, M_prime, new_init_state
    
    def trajectory_probability(self, alpha, beta, T_prime, M_prime, 
new_init_state):
        P_original = sum(alpha[-1,:])
        P_prime = np.array([0.])
        alpha_new = np.zeros((self.Observations.shape[0], self.Transition.shape[0]))
        for i in range(self.Observations.shape[0]):
            sequence_val = self.Observations[i]
            for j in range(self.Transition.shape[0]):
                if (i == 0):
                    alpha_new[i, j] = new_init_state[j] * M_prime[j,sequence_val]
                else:
                    values = [alpha_new[i - 1,k] * M_prime[j,sequence_val] * T_prime[k,j] for k in
                              range(self.Transition.shape[0])]
                    alpha_new[i,j] = sum(values)
        p_prime = np.array(alpha_new[-1,:])
        return P_original, P_prime

In [20]:
k = HMM(np.array([2,0,0,2,1,2,1,1,1,2,1,1,1,1,1,2,2,0,0,1]),np.array([[0.5,0.5],[0.5,0.5]]),np.array([[0.4,0.1,0.5],[0.1,0.5,0.4]]),np.array([0.5,0.5]))

In [37]:
alpha = k.forward()

In [38]:
beta = k.backward()

In [39]:
gamma = k.gamma_comp(alpha,beta)

1.9153478765258798e-10


In [40]:
x = k.xi_comp(alpha,beta,gamma)

In [41]:
[t_prime,m_prime,n_ew_state] = k.update(alpha,beta,gamma,x)

In [42]:
alpha

array([[2.50000000e-01, 2.00000000e-01],
       [9.00000000e-02, 2.25000000e-02],
       [2.25000000e-02, 5.62500000e-03],
       [7.03125000e-03, 5.62500000e-03],
       [6.32812500e-04, 3.16406250e-03],
       [9.49218750e-04, 7.59375000e-04],
       [8.54296875e-05, 4.27148438e-04],
       [2.56289063e-05, 1.28144531e-04],
       [7.68867188e-06, 3.84433594e-05],
       [1.15330078e-05, 9.22640625e-06],
       [1.03797070e-06, 5.18985352e-06],
       [3.11391211e-07, 1.55695605e-06],
       [9.34173633e-08, 4.67086816e-07],
       [2.80252090e-08, 1.40126045e-07],
       [8.40756270e-09, 4.20378135e-08],
       [1.26113440e-08, 1.00890752e-08],
       [5.67510482e-09, 4.54008386e-09],
       [2.04303773e-09, 5.10759434e-10],
       [5.10759434e-10, 1.27689858e-10],
       [3.19224646e-11, 1.59612323e-10]])

In [43]:
beta

array([[4.25632861e-10, 4.25632861e-10],
       [1.70253145e-09, 1.70253145e-09],
       [6.81012578e-09, 6.81012578e-09],
       [1.51336129e-08, 1.51336129e-08],
       [5.04453762e-08, 5.04453762e-08],
       [1.12100836e-07, 1.12100836e-07],
       [3.73669453e-07, 3.73669453e-07],
       [1.24556484e-06, 1.24556484e-06],
       [4.15188281e-06, 4.15188281e-06],
       [9.22640625e-06, 9.22640625e-06],
       [3.07546875e-05, 3.07546875e-05],
       [1.02515625e-04, 1.02515625e-04],
       [3.41718750e-04, 3.41718750e-04],
       [1.13906250e-03, 1.13906250e-03],
       [3.79687500e-03, 3.79687500e-03],
       [8.43750000e-03, 8.43750000e-03],
       [1.87500000e-02, 1.87500000e-02],
       [7.50000000e-02, 7.50000000e-02],
       [3.00000000e-01, 3.00000000e-01],
       [1.00000000e+00, 1.00000000e+00]])

In [44]:
t_prime

array([[0.47023206, 0.52976794],
       [0.3526061 , 0.6473939 ]])

In [45]:
m_prime

array([[0.3902439 , 0.20325203, 0.40650407],
       [0.06779661, 0.70621469, 0.2259887 ]])

In [46]:
n_ew_state

array([0.55555556, 0.44444444])

In [47]:
gamma

array([[0.55555556, 0.44444444],
       [0.8       , 0.2       ],
       [0.8       , 0.2       ],
       [0.55555556, 0.44444444],
       [0.16666667, 0.83333333],
       [0.55555556, 0.44444444],
       [0.16666667, 0.83333333],
       [0.16666667, 0.83333333],
       [0.16666667, 0.83333333],
       [0.55555556, 0.44444444],
       [0.16666667, 0.83333333],
       [0.16666667, 0.83333333],
       [0.16666667, 0.83333333],
       [0.16666667, 0.83333333],
       [0.16666667, 0.83333333],
       [0.55555556, 0.44444444],
       [0.55555556, 0.44444444],
       [0.8       , 0.2       ],
       [0.8       , 0.2       ],
       [0.16666667, 0.83333333]])